### En este notebook se busca testear la tecnologia CUDA para incrementar la velocidad de procesamiento del analisis de sentimiento.

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch
import pandas as pd
import time

c:\Users\Guille\anaconda3\envs\ProyectML\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Se carga un arcahivo csv para el testeo
rest_google_reviews = pd.read_csv("../Datasets/Restaurantes_Google_Reviews.csv",delimiter=",")
rest_google_reviews.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,104959831058524862747,Samuel Benson,1630018420109,4,A pretty seamless experience from top to botto...,NaN,NaN,0x874d9bf9d1db7c85:0xf1c3706a2b497a3
1,100925653611153628883,Emily Probst,1629223016683,5,Great food and great service! Being so close t...,NaN,NaN,0x874d9bf9d1db7c85:0xf1c3706a2b497a3
2,100496729305505499660,Brianna Dungan,1629141770542,5,Pleasant location.,NaN,NaN,0x874d9bf9d1db7c85:0xf1c3706a2b497a3
3,115619895645642399701,Aaron Tim,1630363338649,5,Finally a wingstop in Provo,NaN,NaN,0x874d9bf9d1db7c85:0xf1c3706a2b497a3
4,115522360097670804488,Heather Ann Bruin,1629597692008,5,NaN,NaN,NaN,0x874d9bf9d1db7c85:0xf1c3706a2b497a3


In [4]:
# Se utliza la funcion "is_available" para saber si CUDA esta disponible.
print("CUDA disponible:", torch.cuda.is_available())

CUDA disponible: True


### La libreria pytorch tiene compatibilidad con varias plataformas de procesamiento, en nuestro caso vamos a utlizar CUDA de la empresa NVIDIA para el procesamiento.

In [7]:
# Se guarda en la variable device el nombre de la plataforma de procesamiento.
device = "cuda"

### Se utilizaran dos modelos:

#### finiteautomata/beto-sentiment-analysis:             Este modelo devuelve variables cualitativas "POS" (positiva), "NEG" (negativa) y "NEU" (netrural).

#### nlptown/bert-base-multilingual-uncased-sentiment:   Este modelo devuelve variables cuantitativas del 1 al 5, representando la cantidad de "estrellas" del review. 

***

### finiteautomata/beto-sentiment-analysis

In [8]:
# Carga modelos a la GPU a excepcion del modelo de tokens que no es compatible con CUDA
tokenizer = AutoTokenizer.from_pretrained("finiteautomata/beto-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/beto-sentiment-analysis")
model = model.to(device)
pipe_beto = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0 if str(device) == "cuda" else -1)

In [10]:
# Función para procesar las reviews individualmente.
def sentiment_status(review):
    if review is None:
        return None
    else:
        text = str(review)[:512]
        clasificacion = pipe_beto(text)
        return clasificacion[0]['label']

In [13]:
# Contador de reviews procesados
elementos_procesados = 0

# Tiempo inicial
tiempo_inicial = time.time()

# Itera sobre los reviews y muestra el progreso cada 1 minuto
for index, row in rest_google_reviews.iterrows():
    rest_google_reviews.at[index, 'sentiment'] = sentiment_status(row['text'])
    elementos_procesados += 1
    
    # Calcula el tiempo transcurrido
    tiempo_transcurrido = time.time() - tiempo_inicial
    
    # Calcula el tiempo estimado restante
    elementos_restantes = len(rest_google_reviews) - elementos_procesados
    tiempo_restante = (tiempo_transcurrido / elementos_procesados) * elementos_restantes
    
    # Verifica si ha pasado 1 minuto
    if tiempo_transcurrido >= 60:
        tiempo_inicial = time.time()
        print("************************************************")
        print(f"filas procesadas: {elementos_procesados}/{len(rest_google_reviews)}")
        print(f"% completado: {round(elementos_procesados/len(rest_google_reviews)*100, 2)}%")
        print(f"Tiempo estimado restante: {round(tiempo_restante/60,2)} minutos")

print("Procesamiento completado.")

c:\Users\Guille\anaconda3\envs\ProyectML\lib\site-packages\transformers\pipelines\base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


************************************************
filas procesadas: 7398/152797
% completado: 4.84%
Tiempo estimado restante: 19.65 minutos
************************************************
filas procesadas: 14958/152797
% completado: 9.79%
Tiempo estimado restante: 9.22 minutos
************************************************
filas procesadas: 22481/152797
% completado: 14.71%
Tiempo estimado restante: 5.8 minutos
************************************************
filas procesadas: 29988/152797
% completado: 19.63%
Tiempo estimado restante: 4.1 minutos
************************************************
filas procesadas: 37456/152797
% completado: 24.51%
Tiempo estimado restante: 3.08 minutos
************************************************
filas procesadas: 44918/152797
% completado: 29.4%
Tiempo estimado restante: 2.4 minutos
************************************************
filas procesadas: 52322/152797
% completado: 34.24%
Tiempo estimado restante: 1.92 minutos
**************************

In [15]:
rest_google_reviews.head(10)

,user_id,name,time,rating,text,pics,resp,gmap_id,sentiment
0,104959831058524862747,Samuel Benson,1630018420109,4,A pretty seamless experience from top to botto...,NaN,NaN,0x874d9bf9d1db7c85:0xf1c3706a2b497a3,NEU
1,100925653611153628883,Emily Probst,1629223016683,5,Great food and great service! Being so close t...,NaN,NaN,0x874d9bf9d1db7c85:0xf1c3706a2b497a3,POS
2,100496729305505499660,Brianna Dungan,1629141770542,5,Pleasant location.,NaN,NaN,0x874d9bf9d1db7c85:0xf1c3706a2b497a3,NEU
3,115619895645642399701,Aaron Tim,1630363338649,5,Finally a wingstop in Provo,NaN,NaN,0x874d9bf9d1db7c85:0xf1c3706a2b497a3,NEU
4,115522360097670804488,Heather Ann Bruin,1629597692008,5,NaN,NaN,NaN,0x874d9bf9d1db7c85:0xf1c3706a2b497a3,NEU
5,111139967532022596352,Hootie Hennessey,1629774057722,5,NaN,NaN,NaN,0x874d9bf9d1db7c85:0xf1c3706a2b497a3,NEU
6,116839886208413920152,Andrea Ayala,1630539640545,5,This establishment is very family oriented and...,NaN,"{'time': 1630944387641, 'text': 'Thank you, An...",0x87530f390e058629:0x292320115b1705ea,NEU
7,111930204658217490762,Jeanette Rodgers,1625534118216,4,Wow. Already has three 5-star reviews. Very re...,[{'url': ['https://lh5.googleusercontent.com/p...,"{'time': 1630945915991, 'text': ""Jeanette, Tha...",0x87530f390e058629:0x292320115b1705ea,POS
8,110612871828582311715,Lissette Diaz,1627706899742,5,"Best best best Mexican food. Authentic, colorf...",NaN,"{'time': 1630946322270, 'text': ""Thank you, so...",0x87530f390e058629:0x292320115b1705ea,POS
9,108947133017703821995,Keumi,1627766593988,5,"Absolutely delicious, best Mexican food in tow...",NaN,"{'time': 1630946176765, 'text': ""Keumi, Thank ...",0x87530f390e058629:0x292320115b1705ea,POS


***

### nlptown/bert-base-multilingual-uncased-sentiment

In [16]:
# Cargar el tokenizer y el modelo en el dispositivo
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment').to(device)

In [17]:
# Función para procesar la reseña y mostrar progreso cada 1 minuto
def sentiment_stars(review):
    if review is None:
        return None
    else:
        text = str(review)[:512]
        tokens = tokenizer.encode(text, return_tensors='pt').to(device)
        result = model(tokens)[0]  # Access the logits
        return int(torch.argmax(result))

In [18]:
# Contador de reviews procesados
elementos_procesados = 0

# Tiempo inicial
tiempo_inicial = time.time()

# Itera sobre los reviews y muestra el progreso cada 1 minuto
for index, row in rest_google_reviews.iterrows():
    rest_google_reviews.at[index, 'sentiment'] = sentiment_stars(row['text'])
    elementos_procesados += 1
    
    # Calcula el tiempo transcurrido
    tiempo_transcurrido = time.time() - tiempo_inicial
    
    # Calcula el tiempo estimado restante
    elementos_restantes = len(rest_google_reviews) - elementos_procesados
    tiempo_restante = (tiempo_transcurrido / elementos_procesados) * elementos_restantes
    
    # Verifica si ha pasado 1 minuto
    if tiempo_transcurrido >= 60:
        tiempo_inicial = time.time()
        print("************************************************")
        print(f"filas procesadas: {elementos_procesados}/{len(rest_google_reviews)}")
        print(f"% completado: {round(elementos_procesados/len(rest_google_reviews)*100, 2)}%")
        print(f"Tiempo estimado restante: {round(tiempo_restante/60,2)} minutos")

print("Procesamiento completado.")

************************************************
filas procesadas: 6359/152797
% completado: 4.16%
Tiempo estimado restante: 23.03 minutos
************************************************
filas procesadas: 12513/152797
% completado: 8.19%
Tiempo estimado restante: 11.21 minutos
************************************************
filas procesadas: 19002/152797
% completado: 12.44%
Tiempo estimado restante: 7.04 minutos
************************************************
filas procesadas: 25513/152797
% completado: 16.7%
Tiempo estimado restante: 4.99 minutos
************************************************
filas procesadas: 32029/152797
% completado: 20.96%
Tiempo estimado restante: 3.77 minutos
************************************************
filas procesadas: 38530/152797
% completado: 25.22%
Tiempo estimado restante: 2.97 minutos
************************************************
filas procesadas: 45031/152797
% completado: 29.47%
Tiempo estimado restante: 2.39 minutos
**********************

In [20]:
rest_google_reviews.head(10)

,user_id,name,time,rating,text,pics,resp,gmap_id,sentiment
0,104959831058524862747,Samuel Benson,1630018420109,4,A pretty seamless experience from top to botto...,NaN,NaN,0x874d9bf9d1db7c85:0xf1c3706a2b497a3,4
1,100925653611153628883,Emily Probst,1629223016683,5,Great food and great service! Being so close t...,NaN,NaN,0x874d9bf9d1db7c85:0xf1c3706a2b497a3,4
2,100496729305505499660,Brianna Dungan,1629141770542,5,Pleasant location.,NaN,NaN,0x874d9bf9d1db7c85:0xf1c3706a2b497a3,3
3,115619895645642399701,Aaron Tim,1630363338649,5,Finally a wingstop in Provo,NaN,NaN,0x874d9bf9d1db7c85:0xf1c3706a2b497a3,4
4,115522360097670804488,Heather Ann Bruin,1629597692008,5,NaN,NaN,NaN,0x874d9bf9d1db7c85:0xf1c3706a2b497a3,1
5,111139967532022596352,Hootie Hennessey,1629774057722,5,NaN,NaN,NaN,0x874d9bf9d1db7c85:0xf1c3706a2b497a3,1
6,116839886208413920152,Andrea Ayala,1630539640545,5,This establishment is very family oriented and...,NaN,"{'time': 1630944387641, 'text': 'Thank you, An...",0x87530f390e058629:0x292320115b1705ea,4
7,111930204658217490762,Jeanette Rodgers,1625534118216,4,Wow. Already has three 5-star reviews. Very re...,[{'url': ['https://lh5.googleusercontent.com/p...,"{'time': 1630945915991, 'text': ""Jeanette, Tha...",0x87530f390e058629:0x292320115b1705ea,2
8,110612871828582311715,Lissette Diaz,1627706899742,5,"Best best best Mexican food. Authentic, colorf...",NaN,"{'time': 1630946322270, 'text': ""Thank you, so...",0x87530f390e058629:0x292320115b1705ea,4
9,108947133017703821995,Keumi,1627766593988,5,"Absolutely delicious, best Mexican food in tow...",NaN,"{'time': 1630946176765, 'text': ""Keumi, Thank ...",0x87530f390e058629:0x292320115b1705ea,4
